In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('data_cleaned.csv')
data.shape

(891, 25)

In [3]:
data.head()

,Survived,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,SibSp_0,SibSp_1,...,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Embarked_C,Embarked_Q,Embarked_S
0,0,22.0,7.2500,0,0,1,0,1,0,1,...,1,0,0,0,0,0,0,0,0,1
1,1,38.0,71.2833,1,0,0,1,0,0,1,...,1,0,0,0,0,0,0,1,0,0
2,1,26.0,7.9250,0,0,1,1,0,1,0,...,1,0,0,0,0,0,0,0,0,1
3,1,35.0,53.1000,1,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,0,1
4,0,35.0,8.0500,0,0,1,0,1,1,0,...,1,0,0,0,0,0,0,0,0,1


# Separating dependent and independent variables

In [4]:
x = data.drop('Survived', axis=1)
y = data['Survived']
x.shape, y.shape

((891, 24), (891,))

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_x = scaler.fit_transform(x)
x = pd.DataFrame(scaled_x, columns=x.columns)
x.head()

,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,SibSp_0,SibSp_1,SibSp_2,...,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Embarked_C,Embarked_Q,Embarked_S
0,-0.592481,-0.502445,-0.565685,-0.510152,0.902587,-0.737695,0.737695,-1.465746,1.806421,-0.180125,...,0.560499,-0.390707,-0.314076,-0.075122,-0.067153,-0.075122,-0.03352,-0.482043,-0.307562,0.615838
1,0.638789,0.786845,1.767767,-0.510152,-1.107926,1.355574,-1.355574,-1.465746,1.806421,-0.180125,...,0.560499,-0.390707,-0.314076,-0.075122,-0.067153,-0.075122,-0.03352,2.074505,-0.307562,-1.623803
2,-0.284663,-0.488854,-0.565685,-0.510152,0.902587,1.355574,-1.355574,0.682247,-0.553581,-0.180125,...,0.560499,-0.390707,-0.314076,-0.075122,-0.067153,-0.075122,-0.03352,-0.482043,-0.307562,0.615838
3,0.407926,0.420730,1.767767,-0.510152,-1.107926,1.355574,-1.355574,-1.465746,1.806421,-0.180125,...,0.560499,-0.390707,-0.314076,-0.075122,-0.067153,-0.075122,-0.03352,-0.482043,-0.307562,0.615838
4,0.407926,-0.486337,-0.565685,-0.510152,0.902587,-0.737695,0.737695,0.682247,-0.553581,-0.180125,...,0.560499,-0.390707,-0.314076,-0.075122,-0.067153,-0.075122,-0.03352,-0.482043,-0.307562,0.615838


In [6]:
from sklearn.model_selection import train_test_split as tts
train_x, test_x, train_y, test_y = tts(x, y, random_state=9, stratify=y)
train_x.shape, test_x.shape, train_y.shape, test_y.shape

((668, 24), (223, 24), (668,), (223,))

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [8]:
def blending(model, train_x, train_y, test_x):
    x_train, x_valid, y_train, y_valid = tts(train_x, train_y, random_state=9)
    model.fit(x_train, y_train)
    
    valid_pred = model.predict(x_valid)
    test_pred = model.predict(test_x)
    return valid_pred, y_valid, test_pred

In [9]:
LR=LogisticRegression()
M1_valid, M1_target, M1_test = blending(LR, train_x, train_y, test_x)

In [10]:
#Model 2= Decision Tree
DT=DecisionTreeClassifier()
M2_valid, M2_target, M2_test = blending(DT, train_x, train_y, test_x)

In [11]:
#Model 3 = K Nearest Neighbour
knn=KNeighborsClassifier()
M3_valid, M3_target, M3_test = blending(knn, train_x, train_y, test_x)

In [12]:
valid_prediction = {
              'LR': M1_valid,
              'DT': M2_valid,
              'knn': M3_valid
              }
new_train = pd.DataFrame(valid_prediction)
new_train.head()

,LR,DT,knn
0,1,0,0
1,1,1,1
2,0,0,0
3,1,0,0
4,0,0,0


In [13]:
test_prediction = {
              'LR': M1_test,
              'DT': M2_test,
              'knn': M3_test
              }
new_test = pd.DataFrame(test_prediction)
new_test.head()

,LR,DT,knn
0,0,0,0
1,1,1,1
2,0,1,0
3,0,0,0
4,0,0,0


In [14]:
model1 = KNeighborsClassifier()
model1.fit(new_train, M1_target)
model1.score(new_test,test_y)

0.7399103139013453